In [ ]:
import tarfile

In [ ]:
if not os.path.isfile('donations/P00000001-ALL.csv'):
    if not os.path.isfile('donations/P00000001-ALL.zip'):
        urllib.request.urlretrieve('ftp://ftp.fec.gov/FEC/Presidential_Map/2012/P00000001/P00000001-ALL.zip',
                                   filename = 'donations/P00000001-ALL.zip')
    zipped_data_set = zipfile.ZipFile('donations/P00000001-ALL.zip')
    zipped_data_set.extractall('donations/')